In [ ]:
from openfermion import *
import numpy as np
from pyquil import *
from pyquil.paulis import *
from scipy.optimize import *
from pyquil.api import WavefunctionSimulator, get_qc
from pyquil.api import *
from pyquil.gates import RY, CNOT, MEASURE, H
import openfermion as of
import openfermionpyscf as ofpyscf
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.environ["HOME"], "VQE")))
from qpus_and_qvms_copy import select_available_qpu

### Prepare Hamiltonian using OpenFermion

In [ ]:
# Set molecule parameters
def build_Hamiltonian(bondLen):
    geometry = [("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, bondLen))]
    basis = "sto-3g"
    multiplicity = 1
    charge = 0
    # Perform electronic structure calculations and
    # obtain Hamiltonian as an InteractionOperator
    hamiltonian = ofpyscf.generate_molecular_hamiltonian(
        geometry, basis, multiplicity, charge)

    # Convert to a FermionOperator
    hamiltonian_ferm_op = of.get_fermion_operator(hamiltonian)
    
    # Map to QubitOperator using the JWT
    hamiltonian_jw = of.jordan_wigner(hamiltonian_ferm_op)
    
    # Convert to Scipy sparse matrix
    hamiltonian_jw_sparse = of.get_sparse_operator(hamiltonian_jw)
    mat = hamiltonian_jw_sparse.toarray()
    # Compute ground energy
    eigs, _ = np.linalg.eigh(mat)
    ground_energy = eigs[0]
    return ground_energy, qubitop_to_paulisum(hamiltonian_jw) #returns the actual ground energy and the hamiltonian

In [ ]:
def qubitop_to_paulisum(qubit_op) -> PauliSum: #Convert an OpenFermion QubitOperator to a PyQuil PauliSum.
    paulisum = 0 * PauliTerm("I", 0)  # starts as the zero operator
    for term, coeff in qubit_op.terms.items():
        if term == ():
            paulisum += PauliTerm("I", 0, coeff)
            continue
        # Build the first factor ( carries the coefficient )
        qubit_idx, pauli_char = term[0]
        pterm = PauliTerm(pauli_char, qubit_idx, coeff)
        # Multiply in any further Paulis
        for qubit_idx, pauli_char in term[1:]:
            pterm *= PauliTerm(pauli_char, qubit_idx)
        paulisum += pterm
    return paulisum.simplify()

### UCCSD Ansatz

In [ ]:
def uccsd_fixed_ansatz(theta_val, qubit_arr): #non-parametric UCCSD ansatz
    term_1 = (PauliTerm("Y", qubit_arr[0]) * PauliTerm("X", qubit_arr[1]) * PauliTerm("X", qubit_arr[2]) * PauliTerm("Y", qubit_arr[3]))
    term_2 = (PauliTerm("X", qubit_arr[0]) * PauliTerm("Y", qubit_arr[1]) * PauliTerm("Y", qubit_arr[2]) * PauliTerm("X", qubit_arr[3]))

    evo_A = exponential_map(term_1)
    evo_B = exponential_map(term_2)

    p = Program()
    # Hartree–Fock |1100>
    p += X(qubit_arr[0])
    p += X(qubit_arr[1])

    # e^{-i θ (A-B)/2}
    p += evo_A(-0.5 * theta_val)
    p += evo_B(+0.5 * theta_val)
    return p


### Parametric Sweep Model

In [ ]:
def basis_key(term, nq=4):
    key = ['Z']*nq
    for q, op in term.operations_as_set(): # overwrite X or Y where present
        key[q] = op
    return ''.join(key) #returns basis key

def basis_program_for_theta(key, theta_val):
    p = uccsd_fixed_ansatz(theta_val, opt_qubits).copy()

    ro = p.declare("ro", "BIT", 4)
    for q, op in enumerate(key):
        if op == 'X':
            p += H(q)
        elif op == 'Y':
            p += RX(-np.pi / 2, q)

    for q in range(4):
        p += MEASURE(q, ro[q])

    p.wrap_in_numshots_loop(shots)
    return qc.compile(p)

def energy_for_theta(theta_val):
    E = const_term
    executables = {k: basis_program_for_theta(k, theta_val)
                   for k in basis_groups}

    for key, exe in executables.items():
        bits = qc.run(exe).readout_data["ro"]
        bits = bits.reshape(shots, 4)
        eig  = 1 - 2 * bits

        for term in basis_groups[key]:
            qs = [q for q, _ in term.operations_as_set()]
            prod = np.prod(eig[:, qs], axis=-1).mean()
            E += term.coefficient.real * prod
    return E


In [ ]:
%%time
from tqdm import tqdm 

depth = 3
shots = 100
opt_qubits=[0,1,7,8]
#opt_qubits = [0,1,2,3]
bondlen=.1
const_term = 0.0
num_params = 4*depth
num_points = 1500
basis_groups = {} 

theta_grid=np.linspace(0.0, np.pi, 31)     # 31 points

#qc = get_qc("4q-qvm")
quantum_processor_id = select_available_qpu()
qc = get_qc(quantum_processor_id)#,  endpoint_id="BF16")

ground_energy, Ham = build_Hamiltonian(bondlen)

for t in Ham.terms:
    if t.operations_as_set() == set():
        const_term += t.coefficient.real
    else:
        k = basis_key(t)
        basis_groups.setdefault(k, []).append(t)

print(f"Evaluating {len(theta_grid)} parameter points (no parametric compilation) …")

results = []
for θ in tqdm(theta_grid):
    results.append(energy_for_theta(θ))

results = np.asarray(results)
idx_min  = np.argmin(results)
best_E   = results[idx_min]
best_θ   = theta_grid[idx_min]
error_sq = (best_E - ground_energy) ** 2

print("\n================  GRID-SCAN RESULT  (non-parametric) ================")
print(f"Exact FCI energy       : {ground_energy: .6f}  Ha")
print(f"Lowest scanned energy  : {best_E: .6f}  Ha")
print(f"Parameter θ (rad)      : {best_θ: .5f}")
print(f"Squared error          : {error_sq}")

In [ ]:
static_arr.append(best_E)
print(static_arr)

In [ ]:
static_arr = []

### Looping code for experiments

In [ ]:
energies_arr = []

In [ ]:
%%time

depth = 3
shots = 100
opt_qubits=[0,1,7,8]
#opt_qubits = [0,1,2,3]
#const_term = 0.0
num_params = 4*depth
num_points = 1500
#basis_groups = {} 

theta_grid=np.linspace(0.0, np.pi, 31)     # 31 points

#qc = get_qc("4q-qvm")
quantum_processor_id = select_available_qpu()
qc = get_qc(quantum_processor_id)

bond_lengths = np.linspace(.1,1.5,20)
true_en = []
vqe_en = []

for bond_len in bond_lengths:
    const_term = 0.0
    basis_groups = {}
    
    theta_grid = np.linspace(0.0, np.pi, 31)   # 31 evenly spaced points
    ground_energy, Ham = build_Hamiltonian(bond_len)
    
    for t in Ham.terms:
        if t.operations_as_set() == set():
            const_term += t.coefficient.real
        else:
            k = basis_key(t)
            basis_groups.setdefault(k, []).append(t)
    
    results = []
    for θ in tqdm(theta_grid):
        results.append(energy_for_theta(θ))

    best_idx = int(np.argmin(results))
    vqe_en.append(results[best_idx])
    true_en.append(ground_energy)

In [ ]:
print(vqe_en)
plt.plot(bond_lengths, true_en, color='blue', label = "True energy")
plt.title("QVM performance, Non-Parametric")
plt.scatter(bond_lengths, vqe_en, color='green', label = "depth 2 Parametric VQE energy calculation")
plt.legend()
plt.xlabel('Atomic distance (Angstroms)')
plt.ylabel('Ground state Energy (Hartrees)')
true_en = np.asarray(true_en)
vqe_en = np.asarray(vqe_en)
mse = np.mean((true_en - vqe_en)**2)
print("Mean squared error: ", mse)
    

In [ ]:
energies_arr.append(vqe_en)
print(energies_arr)

In [ ]:
E = np.vstack(energies_arr)            # (n_runs, n_bond_lengths)

mean = E.mean(axis=0)                  # (N,)
lower = mean - E.min(axis=0)           # (N,)   positive!
upper = E.max(axis=0) - mean           # (N,)   positive!

yerr = np.vstack([lower, upper])       # shape (2, N)

print('bond_lengths :', np.shape(bond_lengths))
print('mean         :', mean.shape)
print('yerr         :', yerr.shape)    # should read (2, N)

# ----------------------------------------------------------------------
# 1) plot
# ----------------------------------------------------------------------
plt.figure(figsize=(6,4))

plt.plot(bond_lengths, true_en, label='True energy')

plt.errorbar(bond_lengths, mean,
             yerr=yerr,
             fmt='o',                  # marker (red circle)
             color='green',
             ecolor='green',
             capsize=4,
             elinewidth=1,
             label='depth-3 VQE')

plt.title('QVM performance, Non-Parametric')
plt.xlabel('Atomic distance (Å)')
plt.ylabel('Ground-state energy (Hartree)')
plt.legend()
plt.tight_layout()
plt.show()